\begin{align}
H_\alpha\phi(x)\equiv \bigg(-\frac{d^2}{dx^2}+\alpha x^2\bigg)\phi(x)=\lambda\phi(x),\quad \langle\phi|\phi\rangle=1,
\end{align}

\begin{align}
T\phi(x)=-\frac{d^2}{dx^2}\phi(x),\quad V_\alpha(x)\phi(x)=\alpha x^2\phi(x),\quad H_\alpha(x)=T+V(x).
\end{align}


\begin{align}
\bigg(-\frac{d^2}{dx^2}+\alpha x^2-\lambda\bigg)\phi(x)\equiv F_\alpha(\phi(x))=0,
\end{align}


\begin{align}
F_\alpha(\phi(x))=0,\quad\langle\phi|\phi\rangle=1.
\end{align}

\begin{align}
	\hat{\phi}_{\alpha_{k}}(x) = \sum_{i=1}^{n} a_{i} \phi_{i}(x) .
\end{align}

Because we are solving an eigenvalue problem, we can arrive at a set of different-looking equations, using the same set of judges $\{\psi_i(x)\}_{i=1}^n$ as before. We can simply plug $\hat{\phi}_{\alpha_k}$ into the Schrodinger equation and project both sides onto the judges, writing

\begin{align}
\sum_{i=1}^na_i\langle \psi_j|H_{\alpha_k}|\phi_i\rangle=\lambda\sum_{i=1}^na_i\langle\psi_j|\phi_i\rangle.
\end{align}

Define now two matrices:

\begin{align}
M_{ij}(\alpha)\equiv \langle\psi_j|H_\alpha|\phi_i\rangle,\quad N_{ij}\equiv\langle\psi_j|\phi_i\rangle.
\end{align}

These are both $n\times n$ matrices, and we now have a generalized eigenvalue problem for $\vec{a}$:

\begin{align}
M(\alpha)\vec{a}=\lambda N\vec{a}.
\end{align}

Situationally, this may be quicker to solve than finding the roots of the nonlinear system that results from the "traditional" RBM approach.

As with traditional RBM approaches, this is only helpful if we can evaluate $M(\alpha)$ quickly for different $\alpha$ values. For the HO, this is not too hard: we can write

\begin{align}
\langle\psi_j|H_\alpha|\phi_i\rangle=\langle\psi_j|T|\phi_i\rangle+\alpha\langle \psi_j|x^2|\phi_i\rangle\equiv M_{ij}^{(0)}+\alpha M_{ij}^{(1)}.
\end{align}

Our eigenvalue equation is then

\begin{align}
[M^{(0)}+\alpha M^{(1)}]\vec{a}=\lambda N\vec{a},
\end{align}

and all of $M^{(0)},M^{(1)}$ and $N$ can be precomputed.

In [5]:
import numpy as np

In [15]:
# Second Derivative
# Potential Matrix
# Components 
# Alphas 
# Projecting Functions (Components) 

def second_derivative_matrix(xgrid):
    N = len(xgrid)
    dx = xgrid[1] - xgrid[0]

    # Generate the matrix for the second derivative using a five-point stencil
    main_diag = np.ones(N) * (-5.0 / 2 / dx**2)
    off_diag = np.ones(N - 1) * 4 / 3 / dx**2
    off_diag2 = np.ones(N - 2) * (-1.0 / (12 * dx**2))

    D2 = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(
      off_diag, k=-1) + np.diag(off_diag2, k=2) + np.diag(off_diag2, k=-2)

    return D2

def potential_matrix(xgrid):
    return np.diag(xgrid**2)

def H_creator(alpha, xgrid):
    d2 = second_derivative_matrix(xgrid)
    pot = potential_matrix(xgrid)
    H = d2 + alpha*pot
    return H

def hf_solve(H):
    evals, evects = np.linalg.eigh(H)
    return evals, evects


In [16]:
alphas = [2,3,5,7,10,12,15]
x_max = 10.0
h = 10**(-1)
x = np.arange(-x_max, x_max + h, h)
m = np.zeros((len(alphas), x.shape[0]))

for i in range(len(alphas)):
    alpha = alphas[i]
    H = H_creator(alpha, x)
    evals, evects = hf_solve(H)
    m[i] = evects[0] / np.linalg.norm(evects[0])*np.sign(evects[0][  int(len(x)/2)  ])

[[ 1.59375851e-18  3.54158582e-19  2.98255579e-18 ...  1.69458390e-01
  -0.00000000e+00 -1.76780038e-01]
 [ 9.39734750e-20  7.46981177e-20  1.76799944e-19 ...  2.05298420e-01
   0.00000000e+00 -2.15217046e-01]
 [-1.84513069e-19 -1.18438145e-19 -2.69220837e-19 ... -2.59139604e-01
   0.00000000e+00  2.74505623e-01]
 ...
 [ 4.01084556e-24  3.24174880e-24  4.76079493e-24 ... -3.47653395e-01
   0.00000000e+00  3.77723171e-01]
 [ 3.78575973e-27  3.01439728e-27  3.48417922e-27 ...  3.73360592e-01
  -0.00000000e+00 -4.09661491e-01]
 [ 2.37622016e-30 -2.14249020e-29 -1.37918185e-28 ...  4.05558804e-01
  -0.00000000e+00 -4.51537292e-01]]


In [18]:
U, sigma, Vh = np.linalg.svd(m)
components = 2
reduced_basis = Vh[:components]

reduced_basis = [reduced_basis[i]*np.sign(reduced_basis[i][  int(len(x)/2)  ]) for i in range(len(reduced_basis))]

In [66]:
psi = np.array(reduced_basis)
phi = np.array(reduced_basis)

In [67]:
d2 = second_derivative_matrix(x)
pot = potential_matrix(x)

In [81]:
def M0(psi, phi, d2, i, j):
    inner_product = np.dot(psi[j], phi[i])
    result_vector = np.dot(d2, psi[j])
    M0 = np.dot(result_vector, phi[i])
    return M0

In [82]:
def M1(psi, phi, pot, i, j):
    inner_product = np.dot(psi[j], phi[i])
    result_vector = np.dot(pot, psi[j])
    M1 = np.dot(result_vector, phi[i])
    return M1

In [87]:
compvec = np.zeros(components)
H_hat = np.array([compvec,compvec])
N = np.array([compvec,compvec])

In [84]:
def create_H_hat(alpha, phi, psi, pot, d2):
    for i in range(len(H_hat[0])):
        for j in range(len(H_hat[1])):
            H_hat[i][j] = M0(psi, phi, d2, i, j) + alpha*M1(psi,phi,pot,i,j)
    return H_hat

In [85]:
H_hat = create_H_hat(5, phi, psi, pot, d2)
print(H_hat)

[[190.49400675  35.89065947]
 [ 35.89065947  66.59441513]]


In [91]:
def create_N(psi, phi):
    for i in range(len(N[0])):
        for j in range(len(N[1])):
            N[i,j] = np.dot(psi[j], phi[1])
            print(i, j)
    return N

In [92]:
N = create_N(psi, phi)
print(N)

0 0
0 1
1 0
1 1
[[1.42247325e-16 1.00000000e+00]
 [1.42247325e-16 1.00000000e+00]]


In [ ]:
def solve(N, H_hat):
    